## Repeating the NYC taxi driver dataset study

Source: https://www.databricks.com/blog/2021/04/07/benchmark-koalas-pyspark-and-dask.html

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
import databricks.koalas as ks
import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import pandas as pd
import numpy as np
import time

print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('koalas version: %s' % ks.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.1.5
numpy version: 1.19.5
koalas version: 1.7.0
dask version: 2021.04.1


### Dataset information

In [3]:
url = "../yellow_tripdata_2011-01.parquet"
koalas_data = ks.read_parquet(url)

In [4]:
print("Number of rows:", len(koalas_data))
print()
print(koalas_data.head(20))

Number of rows: 13464997

    VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  improvement_surcharge  total_amount  congestion_surcharge  airport_fee
0          2  2011-01-01 00:10:00   2011-01-01 00:12:00                4            0.0           1               None           145           145             1          2.9    0.5      0.5        0.28           0.0                    0.0          4.18                   NaN          NaN
1          2  2011-01-01 00:04:00   2011-01-01 00:13:00                4            0.0           1               None           264           264             1          5.7    0.5      0.5        0.24           0.0                    0.0          6.94                   NaN          NaN
2          2  2011-01-01 00:14:00   2011-01-01 00:16:00                4            0.0           1           

In [5]:
koalas_data.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [6]:
expr_filter = (koalas_data['tip_amount'] >= 1) & (koalas_data['tip_amount'] <= 5)
 
print(f'Filtered data is {len(koalas_data[expr_filter]) / len(koalas_data) * 100}% of total data')

Filtered data is 35.84216914418919% of total data


### Experiment preparation

In [3]:
def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.

    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name


    Returns
    -------
    Duration (in seconds) of the given operation
    """

    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")

def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

In [4]:
paths = ["../yellow_tripdata_201"+str(i)+"-01.parquet" for i in range(1,4)]

### Koalas

In [27]:
koalas_data = ks.read_parquet(paths[0])

In [28]:
koalas_benchmarks = {
    'duration': [],     # in seconds
    'task': []
}

#### Standard operations

In [29]:
def read_file_parquet(df=None):
    return ks.read_parquet("../yellow_tripdata_2011-01.parquet")

def count(df=None):
    return len(df)

def count_index_length(df=None):
    return len(df.index)

def mean(df):
    return df.fare_amount.mean()

def standard_deviation(df):
    return df.fare_amount.std()

def mean_of_sum(df):
    return (df.fare_amount + df.tip_amount).mean()

def sum_columns(df):
    x = df.fare_amount + df.tip_amount
    return x

def mean_of_product(df):
    return (df.fare_amount * df.tip_amount).mean()

def product_columns(df):
    x = df.fare_amount * df.tip_amount
    return x

def value_counts(df):
    val_counts = df.fare_amount.value_counts()
    return val_counts


#   In the original experiment, the following two functions used the longitude and latitude values of the pickup and the dropout places.
#   Since the datasets provided by NYC TLC Trip Record Data no longer have longitude and latitude values (only the pickup and dropout places IDs),
# we used arbitrary longitude and latitude values. The goal of this experiment is to compare the computational cost of the calculations, hence
# the values are not relevant.
def complicated_arithmetic_operation(df):
    start_lon,end_lon = np.random.randint(-180,180),np.random.randint(-180,180)
    start_lat,end_lat = np.random.randint(-90,90),np.random.randint(-90,90)
    
    theta_1 = start_lon
    phi_1 = start_lat
    theta_2 = end_lon
    phi_2 = end_lat
    temp = (np.sin((theta_2 - theta_1) / 2 * np.pi / 180) ** 2
           + np.cos(theta_1 * np.pi / 180) * np.cos(theta_2 * np.pi / 180) * np.sin((phi_2 - phi_1) / 2 * np.pi / 180) ** 2)
    ret = np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2)
    return ret

def mean_of_complicated_arithmetic_operation(df):
    start_lon,end_lon = np.random.randint(-180,180),np.random.randint(-180,180)
    start_lat,end_lat = np.random.randint(-90,90),np.random.randint(-90,90)
    
    theta_1 = start_lon
    phi_1 = start_lat
    theta_2 = end_lon
    phi_2 = end_lat
    temp = (np.sin((theta_2 - theta_1) / 2 * np.pi / 180) ** 2
           + np.cos(theta_1 * np.pi / 180) * np.cos(theta_2 * np.pi / 180) * np.sin((phi_2 - phi_1) / 2 * np.pi / 180) ** 2)
    ret = np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2) 
    return ret.mean()

def groupby_statistics(df):
    gb = df.groupby(by='passenger_count').agg(
      {
        'fare_amount': ['mean', 'std'], 
        'tip_amount': ['mean', 'std']
      }
    )
    return gb

other = ks.DataFrame(groupby_statistics(koalas_data).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

def join_count(df, other):
    return len(df.merge(other.spark.hint("broadcast"), left_index=True, right_index=True))

def join_data(df, other):
    ret = df.merge(other.spark.hint("broadcast"), left_index=True, right_index=True)
    return ret

In [17]:
benchmark(read_file_parquet, df=None, benchmarks=koalas_benchmarks, name='read file')
benchmark(count, df=koalas_data, benchmarks=koalas_benchmarks, name='count')
benchmark(count_index_length, df=koalas_data, benchmarks=koalas_benchmarks, name='count index length')
benchmark(mean, df=koalas_data, benchmarks=koalas_benchmarks, name='mean')
benchmark(standard_deviation, df=koalas_data, benchmarks=koalas_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=koalas_data, benchmarks=koalas_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=koalas_data, benchmarks=koalas_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=koalas_data, benchmarks=koalas_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=koalas_data, benchmarks=koalas_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=koalas_data, benchmarks=koalas_benchmarks, name='value counts')
benchmark(complicated_arithmetic_operation, df=koalas_data, benchmarks=koalas_benchmarks, name='complex arithmetic ops')
benchmark(mean_of_complicated_arithmetic_operation, df=koalas_data, benchmarks=koalas_benchmarks, name='mean of complex arithmetic ops')
benchmark(groupby_statistics, df=koalas_data, benchmarks=koalas_benchmarks, name='groupby statistics')
benchmark(join_count, koalas_data, benchmarks=koalas_benchmarks, name='join count', other=other)
benchmark(join_data, koalas_data, benchmarks=koalas_benchmarks, name='join', other=other)

read file took: 0.29213380813598633 seconds
count took: 0.23517465591430664 seconds
count index length took: 0.1402449607849121 seconds
mean took: 0.37973570823669434 seconds
standard deviation took: 0.5381433963775635 seconds
mean of columns addition took: 0.6047580242156982 seconds
addition of columns took: 0.029071569442749023 seconds
mean of columns multiplication took: 0.6008937358856201 seconds
multiplication of columns took: 0.029781341552734375 seconds
value counts took: 0.06059908866882324 seconds
complex arithmetic ops took: 0.0 seconds
mean of complex arithmetic ops took: 0.0 seconds
groupby statistics took: 0.13081121444702148 seconds
join count took: 18.390236377716064 seconds
join took: 0.6384189128875732 seconds


#### Operations with filtering

In [30]:
expr_filter = (koalas_data.tip_amount >= 1) & (koalas_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]

koalas_filtered = filter_data(koalas_data)

In [31]:
benchmark(count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered count')
benchmark(count_index_length, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered count index length')
benchmark(mean, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered mean')
benchmark(standard_deviation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered multiplication of columns')
benchmark(mean_of_complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered groupby statistics')

other = ks.DataFrame(groupby_statistics(koalas_filtered).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_data, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered join', other=other)
benchmark(join_count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered join count', other=other)

filtered count took: 1.1444599628448486 seconds
filtered count index length took: 1.096585750579834 seconds
filtered mean took: 1.600801706314087 seconds
filtered standard deviation took: 1.6130115985870361 seconds
filtered mean of columns addition took: 1.2357845306396484 seconds
filtered addition of columns took: 0.053076982498168945 seconds
filtered mean of columns multiplication took: 1.6587965488433838 seconds
filtered multiplication of columns took: 0.05656599998474121 seconds
filtered mean of complex arithmetic ops took: 0.0 seconds
filtered complex arithmetic ops took: 0.0 seconds
filtered value counts took: 0.12258720397949219 seconds
filtered groupby statistics took: 0.33182716369628906 seconds
filtered join took: 1.6664671897888184 seconds
filtered join count took: 36.37161159515381 seconds


#### Operations with filtering and caching

In [32]:
koalas_filtered_cached = koalas_filtered.spark.cache()
print(f'Enforce caching: {len(koalas_filtered_cached)} rows of filtered data')

Enforce caching: 4826147 rows of filtered data


In [33]:
benchmark(count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered count')
benchmark(count_index_length, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered count index length')
benchmark(mean, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered mean')
benchmark(standard_deviation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered multiplication of columns')
benchmark(mean_of_complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered groupby statistics')

other = ks.DataFrame(groupby_statistics(koalas_filtered).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_data, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered join', other=other)
benchmark(join_count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered join count', other=other)

filtered count took: 1.6552259922027588 seconds
filtered count index length took: 1.328669548034668 seconds
filtered mean took: 1.2429137229919434 seconds
filtered standard deviation took: 1.3116066455841064 seconds
filtered mean of columns addition took: 1.3325088024139404 seconds
filtered addition of columns took: 0.0428767204284668 seconds
filtered mean of columns multiplication took: 1.2864344120025635 seconds
filtered multiplication of columns took: 0.04985833168029785 seconds
filtered mean of complex arithmetic ops took: 0.0 seconds
filtered complex arithmetic ops took: 0.0 seconds
filtered value counts took: 0.08036041259765625 seconds
filtered groupby statistics took: 0.28113818168640137 seconds
filtered join took: 1.1048705577850342 seconds
filtered join count took: 15.40094518661499 seconds


### Dask

In [5]:
cluster = LocalCluster(memory_limit='8GB')
client = Client(cluster)

dask_data = dd.read_parquet(paths[0])

In [6]:
dask_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}

#### Standard operations

In [18]:
def read_file_parquet(df=None):
    return dd.read_parquet("../yellow_tripdata_2011-01.parquet")

def count(df=None):
    return len(df)

def count_index_length(df=None):
    return len(df.index)

def mean(df):
    return df.fare_amount.mean().compute()

def standard_deviation(df):
    return df.fare_amount.std().compute()

def mean_of_sum(df):
    return (df.fare_amount + df.tip_amount).mean().compute()

def sum_columns(df):
    return (df.fare_amount + df.tip_amount).compute()

def mean_of_product(df):
    return (df.fare_amount * df.tip_amount).mean().compute()

def product_columns(df):
    return (df.fare_amount * df.tip_amount).compute()

def value_counts(df):
    return df.fare_amount.value_counts().compute()

#   In the original experiment, the following two functions used the longitude and latitude values of the pickup and the dropout places.
#   Since the datasets provided by NYC TLC Trip Record Data no longer have longitude and latitude values (only the pickup and dropout places IDs),
# we used arbitrary longitude and latitude values. The goal of this experiment is to compare the computational cost of the calculations, hence
# the values are not relevant.
def mean_of_complicated_arithmetic_operation(df):
    start_lon,end_lon = np.random.randint(-180,180),np.random.randint(-180,180)
    start_lat,end_lat = np.random.randint(-90,90),np.random.randint(-90,90)
    theta_1 = start_lon
    phi_1 = start_lat
    theta_2 = end_lon
    phi_2 = end_lat
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.mean()

def complicated_arithmetic_operation(df):
    start_lon,end_lon = np.random.randint(-180,180),np.random.randint(-180,180)
    start_lat,end_lat = np.random.randint(-90,90),np.random.randint(-90,90)
    theta_1 = start_lon
    phi_1 = start_lat
    theta_2 = end_lon
    phi_2 = end_lat
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret

def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg({
        'fare_amount': ['mean', 'std'],
        'tip_amount': ['mean', 'std'] 
    })

other = groupby_statistics(dask_data)
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

def join_count(df, other):
    return len(dd.merge(df, other, left_index=True, right_index=True))

def join_data(df, other):
    return dd.merge(df, other, left_index=True, right_index=True).compute()

In [20]:
benchmark(read_file_parquet, df=None, benchmarks=dask_benchmarks, name='read file')
benchmark(count, df=dask_data, benchmarks=dask_benchmarks, name='count')
benchmark(count_index_length, df=dask_data, benchmarks=dask_benchmarks, name='count index length')
benchmark(mean, df=dask_data, benchmarks=dask_benchmarks, name='mean')
benchmark(standard_deviation, df=dask_data, benchmarks=dask_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=dask_data, benchmarks=dask_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=dask_data, benchmarks=dask_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=dask_data, benchmarks=dask_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=dask_data, benchmarks=dask_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=dask_data, benchmarks=dask_benchmarks, name='value counts')
benchmark(mean_of_complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='complex arithmetic ops')
benchmark(groupby_statistics, df=dask_data, benchmarks=dask_benchmarks, name='groupby statistics')
benchmark(join_count, dask_data, benchmarks=dask_benchmarks, name='join count', other=other)
benchmark(join_data, dask_data, benchmarks=dask_benchmarks, name='join', other=other)

read file took: 0.034003257751464844 seconds
count took: 0.358975887298584 seconds
count index length took: 11.157870769500732 seconds
mean took: 1.2081716060638428 seconds
standard deviation took: 1.501474142074585 seconds
mean of columns addition took: 2.0431041717529297 seconds
addition of columns took: 2.578772783279419 seconds
mean of columns multiplication took: 1.9190800189971924 seconds
multiplication of columns took: 2.041478395462036 seconds
value counts took: 0.6384503841400146 seconds
mean of complex arithmetic ops took: 0.011385440826416016 seconds
complex arithmetic ops took: 0.0 seconds
groupby statistics took: 0.04836153984069824 seconds
join count took: 15.860377311706543 seconds
join took: 11.891459941864014 seconds


#### Operations with filtering

In [21]:
expr_filter = (dask_data.tip_amount >= 1) & (dask_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]

dask_filtered = filter_data(dask_data)

In [22]:
benchmark(count, dask_filtered, benchmarks=dask_benchmarks, name='filtered count')
benchmark(count_index_length, dask_filtered, benchmarks=dask_benchmarks, name='filtered count index length')
benchmark(mean, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean')
benchmark(standard_deviation, dask_filtered, benchmarks=dask_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, dask_filtered, benchmarks=dask_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, dask_filtered, benchmarks=dask_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered multiplication of columns')
benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, dask_filtered, benchmarks=dask_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, dask_filtered, benchmarks=dask_benchmarks, name='filtered groupby statistics')

other = groupby_statistics(dask_filtered)
other.columns = pd.Index([e[0] +'_'+ e[1] for e in other.columns.tolist()])

benchmark(join_count, dask_filtered, benchmarks=dask_benchmarks, name='filtered join count', other=other)
benchmark(join_data, dask_filtered, benchmarks=dask_benchmarks, name='filtered join', other=other)

filtered count took: 29.07993721961975 seconds
filtered count index length took: 30.692441701889038 seconds
filtered mean took: 20.941547870635986 seconds
filtered standard deviation took: 17.086211442947388 seconds
filtered mean of columns addition took: 17.045679807662964 seconds
filtered addition of columns took: 13.413687229156494 seconds
filtered mean of columns multiplication took: 10.60387110710144 seconds
filtered multiplication of columns took: 11.928022623062134 seconds
filtered mean of complex arithmetic ops took: 0.02328658103942871 seconds
filtered complex arithmetic ops took: 0.0 seconds
filtered value counts took: 12.697035789489746 seconds
filtered groupby statistics took: 0.2729041576385498 seconds
filtered join count took: 20.698776483535767 seconds
filtered join took: 19.326754093170166 seconds


#### Operations with filtering and caching

In [23]:
dask_filtered = client.persist(dask_filtered)

from distributed import wait
print('Waiting until all futures are finished')
wait(dask_filtered)
print('All futures are finished')

Waiting until all futures are finished
All futures are finished


In [24]:
benchmark(count, dask_filtered, benchmarks=dask_benchmarks, name='filtered count')
benchmark(count_index_length, dask_filtered, benchmarks=dask_benchmarks, name='filtered count index length')
benchmark(mean, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean')
benchmark(standard_deviation, dask_filtered, benchmarks=dask_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, dask_filtered, benchmarks=dask_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, dask_filtered, benchmarks=dask_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered multiplication of columns')
benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, dask_filtered, benchmarks=dask_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, dask_filtered, benchmarks=dask_benchmarks, name='filtered groupby statistics')

other = groupby_statistics(dask_filtered)
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_count, dask_filtered, benchmarks=dask_benchmarks, name='filtered join count', other=other)
benchmark(join_data, dask_filtered, benchmarks=dask_benchmarks, name='filtered join', other=other)

filtered count took: 0.05237889289855957 seconds
filtered count index length took: 0.03733491897583008 seconds
filtered mean took: 0.10164070129394531 seconds
filtered standard deviation took: 0.3326871395111084 seconds
filtered mean of columns addition took: 0.10929036140441895 seconds
filtered addition of columns took: 0.6915786266326904 seconds
filtered mean of columns multiplication took: 0.09549403190612793 seconds
filtered multiplication of columns took: 0.6156878471374512 seconds
filtered mean of complex arithmetic ops took: 0.0 seconds
filtered complex arithmetic ops took: 0.0 seconds
filtered value counts took: 0.10123610496520996 seconds
filtered groupby statistics took: 0.03887438774108887 seconds
filtered join count took: 0.8143243789672852 seconds
filtered join took: 0.3657393455505371 seconds


In [25]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:58707 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 29.80 GiB
